In [1]:
from ko_text import *
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

C:\Users\Dong\AppData\Local\Continuum\anaconda3\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)
C:\Users\Dong\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# **TF-IDF classification**

In [2]:
train_df = pd.read_csv('Data/train_df.csv',
                       usecols = ['Token','Section'],
                       dtype = {'Section' : 'category'},
                       converters = {'Token' : ast.literal_eval},
                     )

test_df = pd.read_csv('Data/test_df.csv',
                       usecols = ['Token','Section'],
                       dtype = {'Section' : 'category'},
                       converters = {'Token' : ast.literal_eval},
                     )

# **Naive Bayes**

In [3]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', OneVsRestClassifier(MultinomialNB(fit_prior=True, class_prior=None))),
])

parameters = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__min_df': (1, 2),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__estimator__alpha': (1e-2, 1e-3)
}

In [ ]:
% time

train_corpus = [' '.join(doc) for doc in train_df['Token']]
test_corpus = [' '.join(doc) for doc in test_df['Token']]

y_train = train_df['Section'].tolist()
y_test = test_df['Section'].tolist()

grid_search_tune = GridSearchCV(pipeline, parameters, cv=2, n_jobs=-1, verbose=3)
grid_search_tune.fit(train_corpus, y_train)

Wall time: 0 ns
Fitting 2 folds for each of 54 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 20.6min


In [ ]:
print("Best parameters set:")
print(grid_search_tune.best_estimator_.steps)

# measuring performance on test set
print("Applying best classifier on test data:")
best_clf = grid_search_tune.best_estimator_
predictions = best_clf.predict(test_df['Token'].tolist())

print(classification_report(test_df['Section'].tolist(), predictions))

# **SVM**

In [6]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', OneVsRestClassifier(LinearSVC())),
])

parameters = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__min_df': (1, 2, 5),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    "clf__estimator__C": [0.01, 0.1, 1],
    "clf__estimator__class_weight": ['balanced', None],
}

# **Logistic Regression**

In [ ]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'))),
])
parameters = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__min_df': (1, 2, 5),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    "clf__estimator__C": [0.01, 0.1, 1],
    "clf__estimator__class_weight": ['balanced', None],
}


# **Grid Search**

In [ ]:
grid_search_tune = GridSearchCV(/
                                pipeline, parameters, cv=2, n_jobs=-1, verbose=3)

grid_search_tune.fit(train_df['Token'].tolist(), train_df['Section'].tolist())


print("Best parameters set:")
print grid_search_tune.best_estimator_.steps

# measuring performance on test set
print "Applying best classifier on test data:"
best_clf = grid_search_tune.best_estimator_
predictions = best_clf.predict(test_x)

print classification_report(test_y, predictions, target_names=genres)